In [2]:
import pandas as pd
import gensim
import multiprocessing
import numpy as np
from utils import pickle_obj, semantic_search_author, semantic_search_word, get_related_authors, get_related_words, translate_dict
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds


In [3]:
df_kth = pd.read_csv("assets/dataframes/all_authors_df_2004")
df_su = pd.read_csv("assets/dataframes/suDf")
df_uppsala = pd.read_csv("assets/dataframes/uppsalaDf")
df_sodertorn = pd.read_csv("assets/dataframes/sodertornDf")

df_kth = df_kth.rename(columns={"KTH_id": "Auth_id", "KTH_name": "Auth_name"})
                    
#df_auth = pd.read_csv("assets/dataframes/KT_auth_2004")

In [4]:
def get_nlp_data(df):
    return df.Abstracts.values, df.Doc_id.values, df.Auth_id.values, df.Auth_name.values
    

In [5]:
text_doc_kth, doc_id_kth, auth_kth, name_kth = get_nlp_data(df_kth)
text_doc_su, doc_id_su, auth_su, name_su = get_nlp_data(df_su)
text_doc_uppsala, doc_id_uppsala, auth_uppsala, name_uppsala = get_nlp_data(df_uppsala)
text_doc_sodertorn, doc_id_sodertorn, auth_sodertorn, name_sodertorn = get_nlp_data(df_sodertorn)

In [6]:
TEXT = np.concatenate([text_doc_kth, text_doc_su, text_doc_uppsala, text_doc_sodertorn])
DOCID = np.concatenate([doc_id_kth, doc_id_su, doc_id_uppsala, doc_id_sodertorn]).astype(str)
AUTHID = np.concatenate([auth_kth, auth_su, auth_uppsala, auth_sodertorn ])
NAME = np.concatenate([name_kth, name_su, name_uppsala, name_sodertorn ])

In [7]:
df = pd.DataFrame(data=list(zip(TEXT, AUTHID, DOCID, NAME)), columns=["Abstracts", "Auth_id", "Doc_id", "Auth_name"])

In [8]:
df

,Abstracts,Auth_id,Doc_id,Auth_name
0,A model for the enhanced water dissociation th...,u16fm297:u13dx9f6,65923903,"Dahlkild, Anders A.:Behm, Marten"
1,Wood from white spruce Picea glauca that had ...,u16k1pmb:u1lkf75c,40520967,"Zhang, Liming:Henriksson, Gunnar"
2,"We consider the moduli space H-g,H-n of n-poin...",u1mv0zlg,41290309,"Bergstrom, Jonas"
3,In this paper a day-ahead planning algorithm f...,u16u3erw:u1naf2f4:u1fjok0u,40129874,"Matevosyan, Julija:Olsson, Magnus:Soder, Lennart"
4,"In this letter, we develop a fixed-point arith...",u1s42xk3:u12s8cr8,61385618,"Johansson, Christopher:Lansner, Anders B."
...,...,...,...,...
140172,Cette contribution décrit l'évolution du mouve...,usodertorn4324,9191_sodertorn,"Östberg, Kjell"
140173,The first part gives a chronological overview ...,usodertorn4324,9204_sodertorn,"Östberg, Kjell"
140174,Det första fallet av aids i Sverige diagnostis...,usodertorn4324,9213_sodertorn,"Östberg, Kjell"
140175,For decades “Swedology” was a rich and polemic...,usodertorn4356,9218_sodertorn,"Östlund, David"


In [28]:
auth_id_to_docids = dict()
auth_id_to_abs = dict()
docids_to_abs = dict()

for a_id, d_id, abstract in zip(list(df.Auth_id.values), list(df.Doc_id.values), list(df.Abstracts.values)):
    
    for a in a_id.split(":"):
        if a in list(auth_id_to_docids.keys()):
            auth_id_to_docids[a].append(d_id)
            auth_id_to_abs[a].append(abstract)
        else:
            auth_id_to_docids[a] = [d_id]
            auth_id_to_abs[a] = [abstract]
            
    if d_id in list(docids_to_abs.keys()):
        docids_to_abs[d_id].append(abstract)
    else:
        docids_to_abs[d_id] = [abstract]
        
        
            

{'65923903': ['A model for the enhanced water dissociation that takes place at the solution/membrane interface in electromembrane processes is presented. The mechanisms behind the enhanced water dissociation are poorly understood and therefore a semi-empirical approach is suggested. The enhanced water dissociation is introduced as a heterogeneous surface reaction similar to the well established Butler-“Volmer law for electrode reactions. In the model there are two parameters that need to be determined through experiments. A 1D diffusion boundary layer problem is presented and solved in order to show that a sufficient rate of water dissociation can be obtained with the model. The advantage of the presented model is that it can easily be incorporated into simulations of electromembrane processes such as electrodialysis, electrodeionization and electropermutation. The influence of the enhanced water dissociation on these processes can then be studied.'],
 '40520967': ["\xa0Wood from white

In [33]:
!ls

 AuthToAbs.ipynb
'Chapter 3 and 4 – Clustering and Semi-supervised learning-Copy1.ipynb'
'Chapter 3 and 4 – Clustering and Semi-supervised learning.ipynb'
 Post-clustering-preparing-data-för-githubPages.ipynb
'Post-processing the final result for the presentation.-Copy1.ipynb'
'Post-processing the final result for the presentation.-Copy2.ipynb'
'Post-processing the final result for the presentation..ipynb'
 __pycache__
 assets
 data.txt
 loadDocVev.ipynb
 prepareDivaDocuments.ipynb
 reformat-divaScrape.ipynb
 semiSupervisedDnn.py
 semisup.py
 trainDoc2vec.py
 trainDocVec-All.ipynb
 trainDocVec.ipynb
 train_model_more_school.py
 uppsala_names.pkl
 utils.py


In [36]:
import pickle
pickle.dump(docids_to_abs, open("/home/ekvall/kthLife/python/assets/dictionaries/authToAbs/docids_to_abs.pkl", 'wb'))
pickle.dump(auth_id_to_abs, open("/home/ekvall/kthLife/python/assets/dictionaries/authToAbs/auth_id_to_abs.pkl", 'wb'))
pickle.dump(auth_id_to_docids, open("/home/ekvall/kthLife/python/assets/dictionaries/authToAbs/auth_id_to_docids.pkl", 'wb'))


In [41]:
auth_id_to_abs['u16fm297']

['A model for the enhanced water dissociation that takes place at the solution/membrane interface in electromembrane processes is presented. The mechanisms behind the enhanced water dissociation are poorly understood and therefore a semi-empirical approach is suggested. The enhanced water dissociation is introduced as a heterogeneous surface reaction similar to the well established Butler-“Volmer law for electrode reactions. In the model there are two parameters that need to be determined through experiments. A 1D diffusion boundary layer problem is presented and solved in order to show that a sufficient rate of water dissociation can be obtained with the model. The advantage of the presented model is that it can easily be incorporated into simulations of electromembrane processes such as electrodialysis, electrodeionization and electropermutation. The influence of the enhanced water dissociation on these processes can then be studied.',
 'We have experimentally and analytically studie

In [103]:
df.to_csv("assets/df_all_schools")